In [12]:
import pandas as pd
import numpy as np

In [13]:
articles_data_path = r'C:\Users\ayush\Documents\Columbia MS DS\2nd_SEM\Applied_ML\AML_Final_Project\Data\articles.csv'
customer_data_path = r'C:\Users\ayush\Documents\Columbia MS DS\2nd_SEM\Applied_ML\AML_Final_Project\Data\customers.csv'
transaction_data_path = r'C:\Users\ayush\Documents\Columbia MS DS\2nd_SEM\Applied_ML\AML_Final_Project\Data\transactions_train.csv'

In [14]:
customers_data = pd.read_csv(customer_data_path)
transaction_data = pd.read_csv(transaction_data_path)
articles_data = pd.read_csv(articles_data_path)



# Preprocessing Customer Data

In [8]:
def preprocess_1():
    merged_data = transaction_data.join(customers_data.set_index('customer_id'), on='customer_id')
    a=merged_data.groupby('customer_id')
    aa=merged_data.groupby(['customer_id', 'article_id'])
    b=a.apply(lambda x: (x.nunique()))
    b=b.drop('customer_id', 1).reset_index()
    bbb = bb.groupby(level=0).max()
    bbb=bbb.reset_index()
    customers_data=customers_data.merge(b[['customer_id', 'article_id']], left_on='customer_id', right_on='customer_id')
    customers_data=customers_data.merge(bbb[['customer_id', 0]], left_on='customer_id', right_on='customer_id')
    customers_data=customers_data.rename(columns = {'article_id':'total_unique_items_bought'})
    customers_data=customers_data.rename(columns = {0:'most_freq_bought_item_count'})


C:\Users\ayush\AppData\Local\Temp/ipykernel_13124/3323159919.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  b=b.drop('customer_id', 1).reset_index()


NameError: name 'bb' is not defined

# Preprocessing Articles

In [4]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
    
def preprocess_2(articles_data):
    n = 3 # top n TF-IDF words

    tfidf = TfidfVectorizer(token_pattern=r"\w+") # no words are left out
    X = tfidf.fit_transform(articles_data['detail_desc'].values.astype('U'))
    ind = (-X.todense()).argpartition(n)[:, :n]
    top_words = pd.Series(list(np.array(tfidf.get_feature_names())[ind]))

    subset_cols = ['article_id','product_type_no','graphical_appearance_no','colour_group_code','perceived_colour_value_id','perceived_colour_master_id','department_no','index_group_no','section_no', 'garment_group_no']
    articles_data = articles_data.loc[:,subset_cols]
    articles_data['prod_desc'] = top_words
    return articles_data

In [15]:
articles_data = preprocess_2(articles_data)

C:\Users\ayush\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [18]:
a_t = pd.merge(articles_data,transaction_data, on = 'article_id')
final = pd.merge(customers_data,a_t, on ='customer_id' )

In [41]:
sub_sample = final.sample(frac=0.001, replace=False, random_state=1)

# Label Encoding

In [42]:
sub_sample['club_member_status'] = sub_sample.club_member_status.astype("category").cat.codes
sub_sample['fashion_news_frequency'] = sub_sample.fashion_news_frequency.astype("category").cat.codes
sub_sample['postal_code'] = sub_sample.postal_code.astype("category").cat.codes

# SubSamping & Train Test Split 

In [43]:
X = sub_sample.drop(['customer_id','article_id','t_dat','prod_desc'],axis = 1)
y = sub_sample['article_id']




from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, shuffle=False)

from sklearn.utils import shuffle
X_train, y_train = shuffle(X_train, y_train, random_state=0)
X_test, y_test = shuffle(X_test, y_test, random_state=0)



In [44]:
# X_train[['club_member_status']] = enc.fit_transform(X_train[['club_member_status']].values.reshape(-1,1))
# X_train[['fashion_news_frequency']] = enc.fit_transform(X_train[['fashion_news_frequency']].values.reshape(-1,1))

# Random Forest

In [46]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.compose import make_column_transformer
import sklearn
from sklearn.metrics import accuracy_score

clf2 = RandomForestClassifier(n_estimators=50, min_samples_split=5,random_state=0,max_samples=200,n_jobs=-1,bootstrap = True)


#l=5000000
#clf2.fit(X_train[:l], y_train[:l])

# label = ['club_member_status','fashion_news_frequency']

#preprocess = make_column_transformer((OrdinalEncoder(), label),
#                                      remainder = "passthrough") 


pipe = sklearn.pipeline.make_pipeline(clf2)
    
pipe.fit(X_train.fillna(0), y_train)





Pipeline(steps=[('randomforestclassifier',
                 RandomForestClassifier(max_samples=200, min_samples_split=5,
                                        n_estimators=50, n_jobs=-1,
                                        random_state=0))])

In [47]:
import gc

# freeing up memory
del X_train
gc.collect()

104

# Predicting

In [48]:
# X_test[['club_member_status']] = enc.transform(X_test[['club_member_status']].values.reshape(-1,1))
# X_test[['fashion_news_frequency']] = enc.transform(X_test[['fashion_news_frequency']].values.reshape(-1,1))
y_pred = pipe.predict(X_test.fillna(0))
print(accuracy_score(y_test, y_pred))

0.011819654942331522


In [51]:
pipe.predict_proba(X_test.fillna(0))

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])